In [24]:
from pyspark import SparkConf, SparkContext
from pyspark.sql.functions import *
sc.stop()
from pyspark.sql import SQLContext
sc =SparkContext()
sqlContext = SQLContext(sc)
import pyspark.sql.functions as func
import sys
from pyspark.sql.functions import countDistinct

In [25]:
#reading in data again
df_new = sqlContext.read.parquet("gs://ds-sean-scrapy/dataflow-analisis-production/plenty/events/product=bbm/product=bbm/aggregated/new/transposed/BBM-STICKER-SEND/")
df_demo = sqlContext.read.parquet("gs://ds-user-demographic/parquet/tastes_mau_1024.parquet")

In [26]:
from pyspark.sql.functions import col


active_base  = df_new.alias('a').join(df_demo.alias('b'),col('b.regid') == col('a.user_id_n')).select([col('time'),col('City'),col('Connection'),col('Country'),col('Device_Brand'),col('Device_Model'),col('New_User'),col('OS_Version'),col('app_name'),col('carrier'),col('license_type'),col('a.platform'),col('sticker_id'),col('sticker_pack_id'),col('user_id_n')]+[col('language'),col('age_group'),col('gender')])

In [27]:
#number of days logged in
active_base = active_base.withColumn('date', concat(df_new.time.substr(1, 10)))
active_base = active_base.withColumn('month', concat(df_new.time.substr(4, 5)))
distinctdate = active_base.groupby('user_id_n').agg(countDistinct('date').alias('numofdays'))

In [28]:
# distinctdate.count()

In [29]:
#consecutive days computation
from pyspark.sql import functions as F

new_base = active_base.groupby("user_id_n").agg(F.collect_list("date").alias("date_list"))

In [30]:
from pyspark.sql.types import FloatType,StringType
from datetime import datetime,timedelta
#function to derive average consecutive days
def con_days(raw):
    raw = list(set(raw))
    n = len(raw)
    raw = sorted(raw, key=lambda d: map(int, d.split('-')))
    no_of_days = 0
    no_of_groups = 0
    group = 0
    total_days = 0
    res = []
    for i in range(0,n-1):
        current_date = datetime.strptime(raw[i], "%Y-%m-%d")
        current_date_mod = datetime.strftime(current_date, "%Y-%m-%d")
        con_date = current_date + timedelta(days=1)
        con_date_mod = datetime.strftime(con_date, "%Y-%m-%d")

        next_date = datetime.strptime(raw[i+1], "%Y-%m-%d")
        next_date_mod = datetime.strftime(next_date, "%Y-%m-%d")
        if con_date_mod == next_date_mod :
            no_of_days += 1
            total_days += 1
            if no_of_days == 1:
                group += 1
                no_of_groups = group

        else:
            no_of_days = 0
    res.append(total_days)
    return total_days

consecutive_days = udf(con_days,StringType())

def con_groups(raw):
    raw = list(set(raw))
    n = len(raw)
    raw = sorted(raw, key=lambda d: map(int, d.split('-')))
    no_of_days = 0
    no_of_groups = 0
    group = 0
    total_days = 0
    res = []
    for i in range(0,n-1):
        current_date = datetime.strptime(raw[i], "%Y-%m-%d")
        current_date_mod = datetime.strftime(current_date, "%Y-%m-%d")
        con_date = current_date + timedelta(days=1)
        con_date_mod = datetime.strftime(con_date, "%Y-%m-%d")

        next_date = datetime.strptime(raw[i+1], "%Y-%m-%d")
        next_date_mod = datetime.strftime(next_date, "%Y-%m-%d")
        if con_date_mod == next_date_mod :
            no_of_days += 1
            total_days += 1
            if no_of_days == 1:
                group += 1
                no_of_groups = group

        else:
            no_of_days = 0
    return no_of_groups

groups = udf(con_groups,StringType())

In [31]:
new_base = new_base.withColumn("Consective_days",consecutive_days('date_list'))
new_base = new_base.withColumn("Consective_groups",groups('date_list'))
new_base = new_base.withColumn("avg_con_days",when((col('Consective_days') != 0)  & (col('Consective_groups') != 0) , round((col('Consective_days')/col('Consective_groups')))).otherwise(0))


In [32]:
# new_base.count()

In [33]:
#sticker packs sent
stickers_sent = active_base.select('user_id_n','sticker_pack_id').groupby('user_id_n').agg(count('sticker_pack_id').alias('sticker_packs_sent'))

In [34]:
#Distinct sticker packs sent 
stickers_distinct_sent = active_base.select('user_id_n','sticker_pack_id').groupby('user_id_n').agg(countDistinct('sticker_pack_id').alias('distinct_sticker_packs_sent'))

In [35]:
# stickers_distinct_sent.count()

In [36]:
# #rocessing sticker type data
# from pyspark.sql.types import StringType
# import re
# sticker_types = sc.textFile("gs://ds-url-catag/stick_bytype/all_stickers/*") \
#     .map(lambda line: line.split(",")).map(lambda line: (line[0],line[1])).toDF(['id','type'])
# remove_quotes = udf(lambda x: re.sub('"','',x), StringType())
# remove_slashes = udf(lambda x: re.sub('\\\\','',x), StringType())
# new_df = sticker_types.select(*[remove_quotes(column).alias(column) for column in sticker_types.columns])
# new_df =  new_df.select(*[remove_slashes(column).alias(column) for column in new_df.columns])
# new_df.dropDuplicates().write.mode('overwrite').parquet("gs://ds-url-catag/stick_bytype/agg_proc_stickertypes/")


In [37]:
#reading in sticker type data
Final_stickers_types = sqlContext.read.parquet("gs://ds-url-catag/stick_bytype/agg_proc_stickertypes/")

In [38]:
# Final_stickers_types.count()

In [39]:
#sticker types used by users
data_join = active_base.join(Final_stickers_types,Final_stickers_types.id== active_base.sticker_pack_id).select([active_base.sticker_pack_id,active_base.user_id_n]+[Final_stickers_types.id,Final_stickers_types.type])

In [40]:
data_join.groupby('type').agg(countDistinct('user_id_n')).show()

+------------+-------------------------+
|        type|count(DISTINCT user_id_n)|
+------------+-------------------------+
|        FREE|                  7463849|
|SUBSCRIPTION|                      257|
|        PAID|                    24766|
|DISCONTINUED|                   206008|
+------------+-------------------------+



In [41]:
#Deriving sticker type variables
#paid
data_join  = data_join.withColumn("is_paid",when(col('type') == 'PAID',data_join.sticker_pack_id).otherwise(None))
data_join  = data_join.withColumn("is_free",when(col('type') == 'FREE',data_join.sticker_pack_id).otherwise(None))
data_join  = data_join.withColumn("is_subscribed",when(col('type') == 'SUBSCRIPTION',data_join.sticker_pack_id).otherwise(None))
data_join  = data_join.withColumn("is_discontinued",when(col('type') == 'DISCONTINUED',data_join.sticker_pack_id).otherwise(None))

In [42]:
sticker_types_aggre = data_join.groupby('user_id_n','is_paid','is_free','is_subscribed','is_discontinued').count()

In [43]:
#replacing strings with numbers for clustering
sticker_types_aggre = sticker_types_aggre.withColumn('is_paid',when(col('is_paid').isNull(),0).otherwise(1))
sticker_types_aggre = sticker_types_aggre.withColumn('is_free',when(col('is_free').isNull(),0).otherwise(1))
sticker_types_aggre = sticker_types_aggre.withColumn('is_subscribed',when(col('is_subscribed').isNull(),0).otherwise(1))
sticker_types_aggre = sticker_types_aggre.withColumn('is_discontinued',when(col('is_discontinued').isNull(),0).otherwise(1))


In [44]:
more_stick_base = sticker_types_aggre.groupBy('user_id_n').agg(func.sum('is_paid').alias('sum_paid'),func.sum('is_free').alias('sum_free'),func.sum('is_subscribed').alias('sum_subs'),func.sum('is_discontinued').alias('sum_discont'))

In [45]:
#join all datasets
data_one = distinctdate.join(new_base,new_base.user_id_n == distinctdate.user_id_n).select([distinctdate.user_id_n,distinctdate.numofdays]+[new_base.avg_con_days])
data_two = data_one.join(stickers_sent,stickers_sent.user_id_n == data_one.user_id_n).select([data_one.user_id_n,data_one.numofdays,data_one.avg_con_days]+[stickers_sent.sticker_packs_sent])
data_three = data_two.join(stickers_distinct_sent,stickers_distinct_sent.user_id_n == data_two.user_id_n).select([data_two.user_id_n,data_two.numofdays,data_two.avg_con_days,data_two.sticker_packs_sent]+[stickers_distinct_sent.distinct_sticker_packs_sent])
data_four = data_three.join(more_stick_base,more_stick_base.user_id_n == data_three.user_id_n).select([data_three.user_id_n,data_three.numofdays,data_three.avg_con_days,data_three.sticker_packs_sent,data_three.distinct_sticker_packs_sent]+[more_stick_base.sum_paid,more_stick_base.sum_free,more_stick_base.sum_subs,more_stick_base.sum_discont])

In [46]:
#write the final file to all the data 
data_four.dropDuplicates().write.mode('overwrite').parquet("gs://ds-url-catag/stick_statistics/derived_features/")